In [1]:
!pip install -q ultralytics
import os, shutil, yaml
import pandas as pd
from sklearn.model_selection import KFold
from ultralytics import YOLO
import torch
import numpy

# === SETUP ===
all_images = []
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/images"):
    if f.endswith(".png"):
        all_images.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/images/{f}")
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/images"):
    if f.endswith(".png"):
        all_images.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/images/{f}")

# Folder for all images
os.makedirs("/kaggle/working/all_images", exist_ok=True)
for file_path in all_images:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        dest_path = os.path.join("/kaggle/working/all_images", file_name)
        shutil.copy2(file_path, dest_path)
    else:
        print(f'File not found: {file_path}')

all_labels = []
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/labels"):
    if f.endswith(".txt"):
        all_labels.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/labels/{f}")
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/labels"):
    if f.endswith(".txt"):
        all_labels.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/labels/{f}")

# Folder for all labels
os.makedirs("/kaggle/working/all_labels", exist_ok=True)
for file_path in all_labels:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        dest_path = os.path.join("/kaggle/working/all_labels", file_name)
        shutil.copy2(file_path, dest_path)
    else:
        print(f'File not found: {file_path}')

df = pd.DataFrame({'image': all_images})
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for i, (train_idx, val_idx) in enumerate(kf.split(df)):
    if i > 2:
        continue

    print(f"Fold {i} de 2")

    os.makedirs(f"/kaggle/working/fold_{i}", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/train", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/val", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/train/images", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/val/images", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/train/labels", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/val/labels", exist_ok=True)


    # === COPY DATA ===
    for idx in train_idx:
        base = df.iloc[idx]["image"]
        shutil.copy(f"/kaggle/working/all_images/{os.path.basename(base)}",
                    f"/kaggle/working/fold_{i}/train/images/{os.path.basename(base)}")
        shutil.copy(f"/kaggle/working/all_labels/{os.path.basename(base).replace('.png', '.txt')}",
                    f"/kaggle/working/fold_{i}/train/labels/{os.path.basename(base).replace('.png', '.txt')}")
        
    for idx in val_idx:
        base = df.iloc[idx]["image"]
        shutil.copy(f"/kaggle/working/all_images/{os.path.basename(base)}",
                    f"/kaggle/working/fold_{i}/val/images/{os.path.basename(base)}")
        shutil.copy(f"/kaggle/working/all_labels/{os.path.basename(base).replace('.png', '.txt')}",
                    f"/kaggle/working/fold_{i}/val/labels/{os.path.basename(base).replace('.png', '.txt')}")
        

    # === CUSTOM DATA.YAML ===
    data_yaml = {
        'train': f"/kaggle/working/fold_{i}/train/images",
        'val': f"/kaggle/working/fold_{i}/val/images",
        'segmentation': "true",
        'names': {0: 'thermal', 1: 'photovoltaic'}
    }

    with open(f"/kaggle/working/fold_{i}/data.yaml", 'w') as f:
        yaml.dump(data_yaml, f)

    # Load YOLOv8 Model
    model = YOLO("yolo11m-seg")
    
    # Train the model
    model.train(
        data= f"/kaggle/working/fold_{i}/data.yaml",
        epochs=100,
        imgsz=512,
        batch=32,
        augment=True,
        project="results",
        name=f'model_{i}',
        exist_ok=True,
        patience=25,
        lr0=1e-3, 
        lrf=0.01
    )


    # predictions
    preds = {
        "img": [],
        "boil": [],
        "pan": [],
    }
    for img in [x for x in os.listdir(f"/kaggle/working/fold_{i}/train/images") if x.endswith(".png")]:

        results = model(f"/kaggle/working/fold_{i}/train/images/{img}")
        result = results[0]

        preds["img"].append(img)

        class_counts = {}
    
        for cls in result.boxes.cls.tolist():
            class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1
    
        class_names = model.names
        named_counts = {class_names[k]: v for k, v in class_counts.items()}
    
        if "boil" in named_counts:
            preds["boil"].append(named_counts["boil"])
        else:
            preds["boil"].append(0)
            
        if "pan" in named_counts:
            preds["pan"].append(named_counts["pan"])
        else:
            preds["pan"].append(0)
    pd.DataFrame(preds).to_csv(f"fold_{i}_train")

    preds = {
        "img": [],
        "boil": [],
        "pan": [],
    }
    for img in [x for x in os.listdir(f"/kaggle/working/fold_{i}/val/images") if x.endswith(".png")]:

        results = model(f"/kaggle/working/fold_{i}/val/images/{img}")
        result = results[0]

        preds["img"].append(img)

        class_counts = {}
    
        for cls in result.boxes.cls.tolist():
            class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1
    
        class_names = model.names
        named_counts = {class_names[k]: v for k, v in class_counts.items()}
    
        if "boil" in named_counts:
            preds["boil"].append(named_counts["boil"])
        else:
            preds["boil"].append(0)
            
        if "pan" in named_counts:
            preds["pan"].append(named_counts["pan"])
        else:
            preds["pan"].append(0)
    pd.DataFrame(preds).to_csv(f"fold_{i}_val")

    shutil.rmtree(f"/kaggle/working/fold_{i}")
        

shutil.rmtree("/kaggle/working/all_images")
shutil.rmtree("/kaggle/working/all_labels")
#torch.cuda.empty_cache()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.1/994.1 kB 16.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Fold 0 de 2


100%|██████████| 43.3M/43.3M [00:00<00:00, 119MB/s]


Ultralytics 8.3.104 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=/kaggle/working/fold_0/data.yaml, epochs=100, time=None, patience=25, batch=32, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=results, name=model_0, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 755k/755k [00:00<00:00, 17.3MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:09<00:00,  1.39s/it]


                   all        422        920      0.901      0.708      0.809      0.499      0.903      0.682      0.783      0.436
               thermal         74         83      0.902      0.675      0.781      0.395      0.915      0.647      0.754      0.344
          photovoltaic        358        837        0.9      0.741      0.837      0.602      0.892      0.717      0.813      0.527


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 10.8ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to results/model_2

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
image 1/1 /kaggle/working/fold_2/train/images/IDqEKfNS.png: 512x512 2 photovoltaics, 17.1ms
Speed: 2.1ms preprocess, 17.1ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 512)

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
image 1/1 /kaggle/working/fold_2/train/images/IDydmvBdIj.png: 512x512 1 thermal, 1 photovoltaic, 17.2ms
Speed: 1.3ms preprocess, 17.2ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 512)

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
image 1/1 /kaggle/working/fold_2/train/images/IDivfKP0Z07BrW.png: 512x512 1 photovoltaic, 17.1ms
Speed: 1.4ms preprocess, 17.1ms inference, 2.4ms postprocess per image at shape (1, 3, 512, 512)

WARNING ⚠️ Model does not suppor